# Extraccion de palabras de Pagina Web para creacion de diccionario

In [1]:

pip install requests bs4 nltk pyphen xlsxwriter langid spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941172 sha256=1f17a9c3ce871b5bd398375f250e17ce185a52b0380f0184c15d9627f174fb2b
  Stored in directory: /root/.cache/pip/wheels/23/c8/c6/eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully built langid


In [2]:
# importacion de librerias

import requests
from bs4 import BeautifulSoup
import nltk
# import csv
import xlsxwriter
import spacy
import spacy.cli
spacy.cli.download("pt_core_news_sm")
# from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
# from nltk.tokenize import RegexpTokenizer
# nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('averaged_perceptron_tagger')
import pyphen
# import langid
import string







✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


## Separacion en silabas

In [3]:
def portuguese_words_in_silabs(word):

  dict_hifenacion = pyphen.Pyphen(lang='pt')

  silab = dict_hifenacion.inserted(word)
  # print(silab)
  return silab
  # si deseo retornar en un nuevo arreglo
  # return sliab.split('-')

## Test de Silabilizado

In [4]:
# listado de lenguajes disponibles
print("lenguajes disponibles: \n", pyphen.LANGUAGES)
# palabra selparada en silabas
silab_word = portuguese_words_in_silabs('vagalume')
print("palabra vagalume en silabas: \n", silab_word)

lenguajes disponibles: 
 {'af_ZA': PosixPath('/usr/local/lib/python3.10/dist-packages/pyphen/dictionaries/hyph_af_ZA.dic'), 'af': PosixPath('/usr/local/lib/python3.10/dist-packages/pyphen/dictionaries/hyph_af_ZA.dic'), 'be_BY': PosixPath('/usr/local/lib/python3.10/dist-packages/pyphen/dictionaries/hyph_be_BY.dic'), 'be': PosixPath('/usr/local/lib/python3.10/dist-packages/pyphen/dictionaries/hyph_be_BY.dic'), 'bg_BG': PosixPath('/usr/local/lib/python3.10/dist-packages/pyphen/dictionaries/hyph_bg_BG.dic'), 'bg': PosixPath('/usr/local/lib/python3.10/dist-packages/pyphen/dictionaries/hyph_bg_BG.dic'), 'ca': PosixPath('/usr/local/lib/python3.10/dist-packages/pyphen/dictionaries/hyph_ca.dic'), 'cs_CZ': PosixPath('/usr/local/lib/python3.10/dist-packages/pyphen/dictionaries/hyph_cs_CZ.dic'), 'cs': PosixPath('/usr/local/lib/python3.10/dist-packages/pyphen/dictionaries/hyph_cs_CZ.dic'), 'da_DK': PosixPath('/usr/local/lib/python3.10/dist-packages/pyphen/dictionaries/hyph_da_DK.dic'), 'da': PosixP

## Clasificacion morfologica

In [5]:
def morfologic_label(word):
  #print('face 1')
  #wordnet.langs
  synsets = wordnet.synsets(word, lang='por')

  #print('fase 2',synsets)

  if synsets:
    #print ("synsets 0.pos: ", synsets[0].pos())
    #print ("synsets 0: ", synsets[0])
    morf = synsets[0].pos()

    if (morf == 'n'):return 'Sustantivo'
    if (morf == 'v'):return 'Verbo'
    if (morf == 'a'):return 'Adjetivo'
    if (morf == 'r'):return 'Adverbio'

  else:
    #word_token = nlp.ad
    # etiquetas_pos = nltk.pos_tag(word, lang='por')
    #print(etiquetas_pos)
    return None

## Test Clasificacion morfologica

In [7]:
#print("idiomas en wordnet \n", wordnet.langs)

morfolologic_label_word = morfologic_label('vagalume')

print("etiqueta morfologica", morfolologic_label_word)


etiqueta morfologica Sustantivo


## Verificacion de Palabra


In [8]:
# def is_portuguese_word(word):
#     # Detectar el idioma principal de la palabra
#     lang, confidence = langid.classify(word)

#     # Verificar si el idioma detectado es portugués y la confianza es suficientemente alta
#     return lang == 'pt' and confidence > 0.5

## Extraccion de palabras

In [11]:
def extract_portuguese_words_2(url):
  response = requests.get(url)
  html = response.text

  soup = BeautifulSoup(html, 'html.parser')
  #print('fase 4')

  # obtengo el texto de la pagina web
  textContent = soup.get_text()

  #print('fase 5' , textContent[:20])

  nlp = spacy.load("pt_core_news_sm")

  doc = nlp(textContent)

  stop_words = set(stopwords.words('portuguese'))
  #portuguese_word = []
  portuguese_dict = []
  count = 1



  for token in doc:
    if not(token.pos_ =='PUNCT' or token.pos_=='SPACE' or token.pos_=='NUM'):
      silab = portuguese_words_in_silabs(token.text)
      #print(f"Palabra: {token.text}, POS Tag: {token.pos_}")
      portuguese_dict.append([count, token.text, silab, token.pos_])
      count = count + 1

  #filtered_words = set(portuguese_dict)

  #complete_dict = list(filtered_words)
  #result = list(filtered_repeated)
  return portuguese_dict



In [12]:
urlTest = 'https://www.dicio.com.br/lista-de-palavras/'
portuguese_dict = extract_portuguese_words_2(urlTest)


print('Primeras 20 palabras: \n ',portuguese_dict[:20])
print('tamaño de diccionario: \n ', len(portuguese_dict))

Primeras 20 palabras: 
  [[1, 'Lista', 'Lis-ta', 'PROPN'], [2, 'de', 'de', 'ADP'], [3, 'Palavras', 'Pa-la-vras', 'PROPN'], [4, 'Dicio', 'Di-cio', 'PROPN'], [5, 'Dicionário', 'Di-ci-o-ná-rio', 'PROPN'], [6, 'Online', 'On-li-ne', 'PROPN'], [7, 'de', 'de', 'ADP'], [8, 'Português', 'Por-tu-guês', 'PROPN'], [9, 'Dicionário', 'Di-ci-o-ná-rio', 'PROPN'], [10, 'Online', 'On-li-ne', 'PROPN'], [11, 'de', 'de', 'ADP'], [12, 'Português', 'Por-tu-guês', 'PROPN'], [13, 'Dicio', 'Di-cio', 'PROPN'], [14, 'Dicionário', 'Di-ci-o-ná-rio', 'PROPN'], [15, 'Online', 'On-li-ne', 'PROPN'], [16, 'de', 'de', 'ADP'], [17, 'Português', 'Por-tu-guês', 'PROPN'], [18, 'Palavras', 'Pa-la-vras', 'PROPN'], [19, 'com', 'com', 'ADP'], [20, 'Lista', 'Lis-ta', 'PROPN']]
tamaño de diccionario: 
  3771


In [ ]:
# def extract_portuguese_words(url):
#   # Obtengo el contenido de la Pasgina Web
#   #print('test Init')
#   response = requests.get(url)
#   #print('fase 1')
#   html = response.text
#   #print('fase 3')


#   # uso de BeautfulSoup para el analisis HTML
#   soup = BeautifulSoup(html, 'html.parser')
#   #print('fase 4')

#   # obtengo el texto de la pagina web
#   textContent = soup.get_text()
#   #print('fase 5')

#   # tokenizar el texto en palagras

#   #tokenizer = RegexpTokenizer(r'\w+')

#   words = word_tokenize(textContent, language='portuguese')
#   #print(words[:20])
#   #print('fase 6')
#   # filtrado de palabras

#   stop_words = set(stopwords.words('portuguese'))


#   #print('fase 7')
#   filtered_words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]

#   filtered_repeated = set(filtered_words);

#   result = list(filtered_repeated)

#   return result





## Test de extraccion

In [ ]:
# urlTest = 'https://www.dicio.com.br/lista-de-palavras/'
# portuguese_words = extract_portuguese_words(urlTest)


# print('Primeras 20 palabras: \n ',portuguese_words[:20])


Primeras 20 palabras: 
  ['zeladoria', 'ócio', 'umidade', 'ventura', 'questionamento', 'joia', 'vício', 'incontinência', 'vereda', 'cognitivo', 'zaragata', 'excludente', 'relevante', 'heroico', 'cobiçapalavras', 'keep', 'consciencioso', 'zero', 'unissex', 'mórbido']


## Exportacion a CSV

In [ ]:
# with open("portuguese_dict.csv", "w") as csvfile:

#     # Crea un escritor CSV
#     writer = csv.writer(csvfile, delimiter=",")

#     # Escribe el arreglo de arreglos en el archivo CSV
#     for row in portuguese_dict:
#         writer.writerow(row)

## Exportacion a Excel

In [ ]:
workbook = xlsxwriter.Workbook("portuguese_dict.xlsx")

# Crea una hoja de cálculo en el libro de trabajo
worksheet = workbook.add_worksheet()

# Escribe los encabezados en la hoja de cálculo

worksheet.write(0, 0, "Numero de palabra")
worksheet.write(0, 0, "Palabra")
worksheet.write(0, 1, "Silabas")
worksheet.write(0, 2, "Etiqueta Morfologica")

# Escribe el arreglo en la hoja de cálculo
for row in range(len(portuguese_dict)):
    for col in range(len(portuguese_dict[0])):
        worksheet.write(row + 1, col, portuguese_dict[row][col])

# Guarda el libro de trabajo de Excel
workbook.close()